In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
import numpy
from numpy.linalg import norm
import heapq

Using TensorFlow backend.


In [2]:
# define documents
docs = ['Well done!','Good work','Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])

In [3]:
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[4, 29], [7, 34], [2, 14], [42, 34], [16], [4], [12, 14], [22, 7], [12, 34], [16, 42, 29, 41]]


In [4]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 4 29  0  0]
 [ 7 34  0  0]
 [ 2 14  0  0]
 [42 34  0  0]
 [16  0  0  0]
 [ 4  0  0  0]
 [12 14  0  0]
 [22  7  0  0]
 [12 34  0  0]
 [16 42 29 41]]


In [5]:
# define the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [6]:
# compile the model
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [7]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 50.000000


# GloVe

### Cosine Similarity Function

In [8]:
def cosine_similarity(a , b):
    numerator = numpy.dot(a , b)
    denominator = norm(a) * norm(b)
    return numerator/denominator

In [9]:
from keras.preprocessing.text import Tokenizer

docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]
[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [10]:
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])

In [11]:
t = Tokenizer()
t.fit_on_texts(docs)
print(t.word_docs)

defaultdict(<class 'int'>, {'done': 2, 'well': 1, 'good': 2, 'work': 3, 'effort': 2, 'great': 1, 'nice': 1, 'excellent': 1, 'weak': 1, 'poor': 2, 'not': 1, 'have': 1, 'could': 1, 'better': 1})


In [12]:
vocab = len(t.word_counts) + 1

In [13]:
encoded_docs = t.texts_to_sequences(docs)
padded_docs = pad_sequences(encoded_docs, maxlen=4, padding="post")

In [15]:
import numpy as np
# load the whole embedding into memory
embeddings_index = dict()
with open('/home/pratik/Desktop/glove.6B/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [20]:
# cosine_similarity(embeddings_index.get('cat') , embeddings_index.get())
scores_list = []
catN = embeddings_index.get('rat')
for iterator in embeddings_index.keys():
#     print(heapq.nlargest(10 , cosine_similarity(catN , embeddings_index.get(iterator))))
    scores_list.append(cosine_similarity(catN , embeddings_index.get(iterator)))

In [21]:
embeddings_index.get('rat')[:10]

array([-0.46285 ,  0.069915,  0.59665 , -1.3745  , -0.55982 ,  0.027237,
        0.32086 , -0.10734 ,  0.17712 , -0.31103 ], dtype=float32)

# Embedding Matrix of vocab
### Matrix of weights only for words in our training data

In [22]:
# create a weight matrix for words in training docs
# 100 is embedding size, if you are using 
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [23]:
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 4, 100)            1500      
_________________________________________________________________
flatten_3 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               102656    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 104,413
Trainable params: 102,913
Non-trainable params: 1,500
_________________________________________________________________
None
Epoch 1/50
10/10 [==============================] - 2s 192ms/step - loss: 0.7287 - acc: 0.5000
Epoch 2/50
10/10 [==============================] - 0s 2ms/step - loss: 0.6564 - acc: 0.7000
Epoch 3/50
10/10 [==============================] - 0s 873us/step - los